In [102]:
import os, errno, operator, re, sys, subprocess, signal
#sys.path.append(os.path.join(wdir, '..', 'flaubert'))
#import flaubert.punkt
from __future__ import division
from IPython.core.debugger import Pdb
import pickle
import ast
import random
import numpy as np
import enchant
import matplotlib.pyplot as plt
from nltk import bigrams,ConditionalFreqDist,FreqDist,pos_tag,pos_tag_sents
from gensim import parsing, matutils, interfaces, corpora, models, similarities, summarization
from gensim.utils import lemmatize
from nltk import collocations, association, text, tree
from gensim.corpora.mmcorpus import MmCorpus
from gensim.matutils import corpus2csc
from gensim.similarities.docsim import SparseMatrixSimilarity
from reader import Json100CorpusReader
import itertools, shutil
from collections import Counter
# from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
#from sklearn.naive_bayes import MultinomialNB,BernoulliNB
#from sklearn.svm import SVC, LinearSVC
#from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
#from sklearn.pipeline import Pipeline
#from sklearn.grid_search import GridSearchCV
#from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
#from sklearn.tree import DecisionTreeClassifier 
#from sklearn.learning_curve import learning_curve

import re
from lxml import etree
from StringIO import StringIO
from os import listdir
import json
import pandas as pd
from collections import defaultdict

from nltk.grammar import DependencyGrammar
from nltk.parse import (
    DependencyGraph, ProjectiveDependencyParser, NonprojectiveDependencyParser)
from nltk.parse.malt import MaltParser as MaltParser

from nltk.corpus import dependency_treebank as dt
from nltk.corpus import treebank_raw
from nltk.corpus import treebank
from pickle import load
from nltk.parse import stanford
from nltk.corpus.util import LazyCorpusLoader

from glob import iglob
from StringIO import StringIO
from os import listdir
from os.path import getmtime, join, realpath
import getpass

In [103]:
tla = ['abo', 'sub', 'apa', 'cto']
NAME = os.environ.get('NAME') or 'dmoz'
wdir = os.path.expanduser('~/scrapy')
os.environ['CLASSPATH'] = join(wdir, 'lib')
odir = join(wdir, NAME)
os.chdir(odir)
tm_marker1 = getmtime(join(odir, 'marker1'))
lookback = 2
jsons = [f for f in os.listdir(odir) if re.search(r'\.json$', f) and getmtime(f) >= tm_marker1 - 86400*lookback]
craigcr = Json100CorpusReader(odir, jsons)
coords = craigcr.coords()
links = craigcr.links()


In [104]:
def make_dict(crlist):
    stoplist = set('for a of the and to in'.split())
    stoplist = set()
    class nouner(object):
        def __init__(self, cr):
            self.cr = cr
        def __iter__(self):
            for doc in self.cr.sents():
                yield doc
                # for t in list(parser.parse(sent)):
                #     for p in t.productions():
                #         if p.is_lexical():
                #             if p.lhs() == 'NN':
                #                 yield 'NN'
                #             else:
                #                 print p.rhs()
                #                 yield p.rhs()
    # corpora.Dictionary is a static method of gensim.corpora
    # it establishes the base of operations numbering the vocab,
    # and you feed it strings such as doc2bow(feedit) produces a sparse vector.
    dictionary = corpora.Dictionary(list(itertools.chain.from_iterable([list(nouner(cr)) for cr in crlist])))
    checker = enchant.Dict("en_US")
    stop_ids = [dictionary.token2id[stopword] for stopword in stoplist 
                if stopword in dictionary.token2id]
    once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.iteritems() if docfreq == 1]
    mispelled_ids = [dictionary.token2id[word] for word in dictionary.values() if not checker.check(word)]
    dictionary.filter_tokens(stop_ids + once_ids + mispelled_ids)
    
    myset = set(['WRB', 'WP$', 'WP', 'WDT', 'TO', 'RP', 'PRP$', 'PRP', 'PDT', 'MD', 'IN', 'EX', 'DT', 'CC', 'UH', ])
    myset = set(['WRB', 'WP$', 'WP', 'WDT', 'TO', 'RP', 'PRP$', 'PRP', 'PDT', 'MD', 'IN', 'EX', 'DT', 'CC', 'RB', 'RBR', 'RBS', 'UH', 'NNP', 'NNS',  ])
    myset = set(['WRB', 'WP$', 'WP', 'WDT', 'TO', 'RP', 'PRP$', 'PRP', 'PDT', 'MD', 'IN', 'EX', 'DT', 'CC', 'RB', 'RBR', 'RBS', 'UH', 'NNP', 'NNS',  ])
    myset = set(['WRB', 'WP$', 'WP', 'WDT', 'TO', 'RP', 'PRP$', 'PRP', 'PDT', 'MD', 'IN', 'EX', 'DT', 'CC', 'UH', 'RB', 'RBR', 'RBS', 'NN', 'NNP', 'NNS', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBZ' ])
    contextual_ids = [dictionary.token2id[k[0][0]] for k in pos_tag_sents([[w] for w in dictionary.values()]) if k[0][1] not in myset]
    
#    dictionary.filter_tokens(contextual_ids)
    dictionary.compactify()
    return dictionary

In [105]:
class i2what(object):
    def __init__(self, arr):
        self._i2w = arr
    def __len__(self):
        return len(self._i2w)    
    def q_dupe(self, i):
        return self._i2w[i] < 0
    def __getitem__(self, i):
        return abs(self._i2w[i])
    def __iter__(self):
        for i in self._i2w:
            if self.q_dupe(i):
                next
            else:
                yield self[i]
        
def CorpusDedupe(cr, dict):
    corpus = [dict.doc2bow(doc[:30]) for doc in list(cr)]
    i2text = np.arange(1,len(corpus)+1,1)
    i2loc = np.arange(1,len(corpus)+1,1)
    index = SparseMatrixSimilarity(corpus, num_features=len(dict.keys()))
    for i, z in enumerate(zip(index, coords)):
        if i2text[i] > 0:
            negated = -i2text[i]
            for j, sim in enumerate(z[0][i+1:]):
                if sim > .99:
		    i2text[i] = i2text[i+1+j] = negated
        if i2loc[i] > 0 and None not in z[1]:
            ci = z[1]
            negated = -i2loc[i]
            for j, cj in enumerate(coords[i+1:]):
                if ci == cj:
                    i2loc[i] = i2loc[i+1+j] = negated
    return i2what(i2text), i2what(i2loc)


In [106]:
# establish vocab numbering
dict = make_dict([craigcr])
# corpus = collection of bow sparse vectors
i2text, i2loc = CorpusDedupe(craigcr, dict)


In [107]:
from math import radians, sin, cos, sqrt, asin
def haversine(lat1, lon1, lat2, lon2):
    R = 6372.8 # Earth radius in kilometers
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))
 
    return R * c

def within(coords):
    if coords[0] is None or coords[1] is None:
	return True
    if NAME == "dmoz":
	return float(coords[0]) < 40.796126
    # que
    elif NAME == "que":
	km = haversine(40.748604, -73.937523, float(coords[0]), float(coords[1]))
	return km < 4
    # sf
    elif NAME == "sfc":
	km = haversine(37.779076, -122.397501, float(coords[0]), float(coords[1]))
	return km < 1.5
    # berkeley
    elif NAME == "eby":
	km = haversine(37.871454, -122.298115, float(coords[0]), float(coords[1]))
	return km < 3
    # milbrae
    #    km = haversine(37.600122, -122.386914, float(coords[0]), float(coords[1]))
    # daly city
    #    km2 = haversine(37.687915, -122.472452, float(coords[0]), float(coords[1]))
    return True


In [108]:
def filterByWord(vOfv, listedby):
    scam = re.compile("!!+|leasebreak|dalia|greystar", re.IGNORECASE)
    pronouns = re.compile("^(i|me|mine|our|he|she|they|their|we|my|his|her|myself|himself|herself|themselves)$", re.IGNORECASE)
    ok = bool(listedby) # if sublet (not listedby) then default to suspicious
    for w in [w for sent in vOfv for w in sent]:
	if scam.search(w):
	    return False
	if pronouns.search(w):
	    ok = True
    return ok

In [109]:
def nPara(raw):
    lines = raw.split('\n')
    wstart = next( (i for (i,l) in enumerate(lines) if re.search(r"\S", l) ), 0)
    wend = len(lines) - next( (i for (i,l) in enumerate(reversed(lines)) if re.search(r"\S", l) ), 0)
    result = 0
    inPara = False
    for l in lines[wstart:wend]:
        if re.search(r"\S", l):
            if not inPara:
                inPara = True
                result += 1
        elif inPara:
            inPara = False
    return result
    
def numSents(vOfv):
    return len(vOfv)

def numYell(vOfv):
    return sum([1 for v in vOfv for w in v if re.search("[A-Z]{3}", w)])

def numWords(vOfv):
    return sum([len(v) for v in vOfv])

def numNonAscii(vOfv):
    return sum([1 for v in vOfv for w in v if any(ord(char) > 127 for char in w)])



In [112]:
from collections import Counter
listedby = craigcr.attrs_matching(r'[lL]isted')
listedby = [re.split(r':\s*', i, 1).pop() if i else None for i in listedby]
oklistedby = set()
for pair in Counter(listedby).iteritems():
    if not pair[0]:
	oklistedby.add(pair[0])
    elif pair[1] == 1:
	if not re.search(r"no fee|contact|apartments|apts|for all|llc|to view|===+|----+|\*\*\*+|\.\.\.+|xxxx+|best|mark a|marc a|rentals|real|estate", pair[0], re.IGNORECASE):
	    if len(pair[0].split()) < 5:
		oklistedby.add(pair[0])

In [114]:
odoms = craigcr.attrs_matching(r'[oO]dom')
odoms = [re.split(r':\s*', i, 1).pop() if i else None for i in odoms]


In [158]:
try:
    shutil.rmtree(join(odir, 'files'))
except OSError as e:
    if e.errno != errno.ENOENT:
        raise
try:
    os.makedirs(join(odir, 'files'))
except OSError as e:
    if e.errno != errno.EEXIST:
        raise
lst,rej = [],[]


In [159]:
for i,z in enumerate(zip(craigcr.docs(), craigcr.raw(newlines_are_periods=True))):
    listing = '{} {} {}'.format(' '.join([word.encode('utf-8') for sent in z[0] for word in sent][0:50]), \
                                links[i], \
                                re.sub(r'\s+', ' ', listedby[i]) if listedby[i] else "Actual Person?")
    # filter in order of increasing time complexity
    if i2text.q_dupe(i):
        rej.append("dupe {}".format(listing))
        continue
    if listedby[i] not in oklistedby:
        rej.append("listedby {}".format(listing))
        continue
    if odoms[i] and int(odoms[i]) > 160000:
        rej.append("miles {}".format(listing))
        continue
    if not within(coords[i]):
        rej.append("toofar {}".format(listing))
        continue
    if not filterByWord(z[0], listedby[i]):
        rej.append("byword {}".format(listing))
        continue
        
    nw=numWords(z[0])
    ns=numSents(z[0])
    wps=float(nw/ns) if ns else 0.0
    np=nPara(z[1])
    spp=float(len(z[0])/np) if np else 0.0
    yr=float(numYell(z[0])/nw) if nw else 0.0
    nna=numNonAscii(z[0])

    if nna > 3 or wps <= 10 or spp <= 1.0 or yr > 0.1:
        rej.append("garbage {}".format(listing))
        continue
    lst.append("{} np={} wps={} spp={:.2f}".format(listing, np, int(wps), spp))
    tla_link = re.findall(r"({0})/(?:[^/]+/)*?(\d+).html".format('|'.join(tla)), links[i])[-1]
    with open(join(odir, "files", "{}-{}".format(tla_link[0], tla_link[1])), "w") as f:
        f.write(z[1].encode('utf-8'))


In [160]:
def dump(file, tla, lines):
    mark = {}
    re_html = r"{0}/(?:[^/]+/)*?(\d+).html"
    for type in tla:
        try:
            with open(join(odir, 'marker0'), 'r') as f0:
                mark[type] = max(['0'] + [re.findall(re_html.format(type), line)[-1] for line in f0.readlines() if re.findall(re_html.format(type), line)])
        except (IOError, IndexError) as e:
            mark[type] = '0'

    with open(file, 'w+') as f1:
        for type in tla:
            for listing in lines:
                try:
                    id = re.findall(re_html.format(type), listing)[-1]
                    if id > mark[type]:
                        f1.write(listing + '\n\n')
                except IndexError as e:
                    pass    
dump(join(odir, 'digest'), tla, lst)
dump(join(odir, 'reject'), tla, rej)


In [161]:
clientcmd = "gradle -p {} client -Pcargs=\"['-file', '{}', '-outputDirectory', '{}']\"".format(join(wdir, "..", "CoreNLP"), join(odir, "files"), join(odir, "files"))
servercmd = "gradle -p {} server".format(join(wdir, "..", "CoreNLP"))
if getpass.getuser() != 'ubuntu':
    subprocess.Popen(servercmd, shell=True, stdout=subprocess.PIPE, preexec_fn=os.setsid)
    subprocess.Popen(clientcmd, shell=True, stdout=subprocess.PIPE)
# os.system("wget localhost:9000/shutdown?key=$(cat /var/tmp/corenlp.shutdown) -O -")